# Welcome To The Hands On Experience On A Simple Neural Network With Deep Learning (For Image Classifications)

All Literature about PyTorch could be found at [PyTorch](http://pytorch.org/) -http://pytorch.org

###### **To Slides: What is tensor

In [ ]:
#This is how you can comment in jupyter

#Following two commands are needed only if you run your python codes in jupyter notebooks.
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Now we import needed packages
import numpy as np  # does all the numerical calculations 
import torch
import matplotlib.pyplot as plt #library easy to plot a matrix into a graphical plot
from torch import nn #neural network module 
from torch import optim # for the optimization 
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models

 ##### **** Let's start making a convolutional neural network 

There are many datasets available in Torch Vision

A list of such databases are given in --> https://pytorch.org/docs/stable/torchvision/datasets.html

In [ ]:
# Define a transform image augmentation to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                              ])


# Download and load the training data
#We need to define a set and loader. The set defines training validating and test sets and it's boundaries. 
#Loaders actually load data into the memory.
trainset = datasets.CIFAR10('data', download=True, train=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=1)

In [ ]:
#Define your classes 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

##### Converting an images to a Tensor file and Numpy and 

In [ ]:
#itterate function to go through pictures
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:
labels

In [ ]:
images

In [ ]:
#To convert a Tensor to a numpy thereby to visualize the image...
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images in a grid 
imshow(torchvision.utils.make_grid(images))

# print respective labels
print(' '.join('%5s' % classes[labels[j]] for j in range(10)))

###### Let's define a Convolutional Neural Network

In [ ]:

class myNeuralNetwork(nn.Module):
    def __init__(self):
        super(myNeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.relu1=nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


myNeuralNetwork = myNeuralNetwork()

In [ ]:
print(myNeuralNetwork)

In [ ]:
#Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(myNeuralNetwork.parameters(), lr=0.001)

In [ ]:
#Train the network
#epoch is Means an itterator. 

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): #this is how to write a for loop in python. 
                                              #simillar to i;i=0;i++ in c++ or java.
            
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Now we do the training.
        #1. First goes the forward pass. Get the output.
        #we difine a loss function (so as to calculate how much far is the predicted from the actual)
        #2. Then the output is compared against the actual.
        #3. THen the backward propergation is made to optimize the parameters to get a minimum loss.
        #Minimum loss means the prediction is so close to the actual label.        
        outputs = myNeuralNetwork(inputs)
        loss = criterion(outputs, labels)
        loss.backward()     
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
#Now the training is done. 
#Let's test on test data.

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
#machine learning prediction
outputs = myNeuralNetwork(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = myNeuralNetwork(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy for 10,000 images is: %d %%' % (
    100 * correct / total))

In [ ]:
?torch.no_grad

In [ ]:
predicted

That looks way better than chance, which is 10% accuracy (randomly picking a class out of 10 classes). Seems like the network learnt something.

Hmmm, what are the classes that performed well, and the classes that did not perform well:

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = myNeuralNetwork(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))